# Exploration Initiale : Station P

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (15, 6)

## 1. Chargement des données

In [ ]:
file_path = 'bronze/bronze/sensor_P_20231110_20260106.csv'

df = pd.read_csv(file_path)

df['ts'] = pd.to_datetime(df['ts'])

print(f"Nombre total de lignes : {len(df)}")
display(df.head())

In [ ]:
sensor_counts = df['sensor'].value_counts()
print("Nombre de mesures par capteur :")
print(sensor_counts)

In [ ]:
import plotly.express as px

fig = px.scatter(
    df, 
    x='ts', 
    y='sensor', 
    color='sensor',
    title="Frise chronologique interactive (Station P)",
    render_mode='webgl'
)

fig.update_traces(marker=dict(size=3, opacity=0.8))

fig.show()

In [ ]:
summary = df.groupby('sensor')['ts'].agg(['min', 'max'])
summary['duree'] = summary['max'] - summary['min']

display(summary)

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('bronze/bronze/sensor_P_20231110_20260106.csv')
df['ts'] = pd.to_datetime(df['ts'])


# On utilise 'first' car à une minute donnée, on a une seule valeur par capteur
df_pivot = df.pivot_table(index='ts', columns='sensor', values='value', aggfunc='first')

# On remplit les trous (NaN) par la méthode "Forward Fill" (on garde la dernière valeur connue)
# au cas où un capteur saute une minute
df_pivot = df_pivot.ffill()

#valeurs négatives à 0
if 'entry_debit_f1' in df_pivot.columns:
    df_pivot['entry_debit_clean'] = df_pivot['entry_debit_f1'].clip(lower=0)
else:
    print(" Pas de colonne entry_debit_f1")

#échantillon 2j
start_date = '2024-07-07'
end_date = '2024-07-09'

mask = (df_pivot.index >= start_date) & (df_pivot.index <= end_date)
df_zoom = df_pivot.loc[mask]

fig = px.line(df_zoom, y=['entry_debit_f1', 'entry_debit_clean'], 
              title='Analyse du profil de la Pompe (Brut vs Nettoyé)',
              labels={'value': 'Débit (m3/h)', 'ts': 'Temps'})
fig.show()

Question : La pompe essaie de verser de l'eau à mesure que l'eau entre ? mais pas en continu, seulement quand il ya un certain niveau d'eau, c'est pour ça que des fois le débit est à 0 (alors qu'en vrai c'est pas le cas) ?

Et pourquoi des fois débit négatif? --> toujours en soirée!

## Volume entrant (Ville) = Volume sortant (Pompe)

In [ ]:
from scipy.signal import find_peaks
import numpy as np

#sur échantillon zoomé d'abord 
series = df_zoom['entry_debit_clean']

#pr trouver les pics:
# height=20 : on ne garde que les pics supérieurs à 20 m3/h
# distance=5 : on impose qu'il y ait au moins 5 minutes entre deux pics
peaks_indices, _ = find_peaks(series, height=20, distance=5)
peak_times = series.index[peaks_indices]

#différence de temps entre chaque pic consécutif
cycle_durations = pd.Series(peak_times).diff().dropna()
cycle_durations_min = cycle_durations.dt.total_seconds() / 60

print(f"Nombre de cycles détectés sur la période : {len(cycle_durations)}")
print(f"Durée minimale d'un cycle : {cycle_durations_min.min():.1f} minutes")
print(f"Durée MOYENNE d'un cycle : {cycle_durations_min.mean():.1f} minutes")
print(f"Durée MAXIMALE d'un cycle : {cycle_durations_min.max():.1f} minutes")

fig = px.line(df_zoom, y='entry_debit_clean', title="Détection des cycles de pompage")
fig.add_scatter(x=peak_times, y=series.iloc[peaks_indices], mode='markers', name='Pics détectés', marker=dict(color='red', size=8))
fig.show()

Dans l'idéal : fenêtre de lissage supérieure à la durée du cycle le plus long (Temps de remplissage + Temps de vidange par temps sec).
On avait dit : pas de prédiction en dessous d'un horizon de temps de 3h -> on choisit une window_size de 3h.

In [ ]:
window_size = 180  # 3h

# Calcul de la moyenne glissante
df_zoom['entry_debit_smooth_3h'] = df_zoom['entry_debit_clean'].rolling(window=window_size, center=True).mean()
fig = px.line(df_zoom, y=['entry_debit_clean', 'entry_debit_smooth_3h'], 
              title=f'Lissage adapté aux cycles longs (Fenêtre de {window_size} min)',
              labels={'value': 'Débit (m3/h)', 'ts': 'Temps'},
              color_discrete_map={'entry_debit_clean': 'orange', 'entry_debit_smooth_3h': 'green'})

fig.update_traces(selector=dict(name='entry_debit_smooth_3h'), line=dict(width=4))
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks

df['ts'] = pd.to_datetime(df['ts'])

df_pivot = df.pivot_table(index='ts', columns='sensor', values='value', aggfunc='first')

# On ne garde que la période où le débit existe
df_full = df_pivot.dropna(subset=['entry_debit_f1']).copy()
df_full = df_full.ffill()
df_full['entry_debit_clean'] = df_full['entry_debit_f1'].clip(lower=0)
print(f"Période traitée : du {df_full.index.min()} au {df_full.index.max()}")
series = df_full['entry_debit_clean']
peaks_indices, _ = find_peaks(series, height=20, distance=5)
cycle_durations = pd.Series(series.index[peaks_indices]).diff().dropna().dt.total_seconds() / 60

print(f"Durée MOYENNE d'un cycle : {cycle_durations.mean():.1f} min")
print(f"Durée MAXIMALE d'un cycle : {cycle_durations.max():.1f} min")

# 4. Lissage global (3h)
window_size = 180
df_full['entry_debit_smooth_3h'] = df_full['entry_debit_clean'].rolling(window=window_size, center=True).mean()

# On sous-échantillonne (1 point toutes les 10 min) pour ne pas faire planter le graphique
df_plot = df_full.iloc[::10, :] 

fig_raw = px.line(df_plot, y='entry_debit_clean', 
              title='Vérification : Débit Brut Nettoyé (Sans valeurs négatives)',
              labels={'entry_debit_clean': 'Débit Pompe (m3/h)', 'ts': 'Temps'},
              render_mode='webgl')
fig_raw.update_traces(line=dict(color='orange', width=1))
fig_raw.show()

fig_smooth = px.line(df_plot, y='entry_debit_smooth_3h', 
              title=f'Résultat Final : Débit Lissé sur 3h (Target du modèle)',
              labels={'entry_debit_smooth_3h': 'Débit Moyen (m3/h)', 'ts': 'Temps'},
              render_mode='webgl')
fig_smooth.update_traces(line=dict(color='green', width=2))
fig_smooth.show()

output_path = 'bronze/bronze/sensor_P_clean.csv'
cols_to_drop = ['aeration_f1', 'oxygen_f1', 'redox_f1']
df_save = df_full.drop(columns=cols_to_drop, errors='ignore')

print(f"Colonnes conservées : {list(df_save.columns)}")

df_save.to_csv(output_path)


Question : pourquoi le débit est beaucoup plus important à partir de avril 2025 ? Travaux?
--> les données de météo commencent le 1er janvier 2015 et s'arrêtent le 31 décembre 2024. On ne garde donc que les données de l'overlap entry_debit_f1 et données méteo.

In [ ]:
import pandas as pd
import plotly.express as px
from scipy.signal import find_peaks

df['ts'] = pd.to_datetime(df['ts'])

print("Pivot de la table en cours...")
df_pivot = df.pivot_table(index='ts', columns='sensor', values='value', aggfunc='first')

df_full = df_pivot.dropna(subset=['entry_debit_f1']).copy()
df_full = df_full.ffill()
df_full['entry_debit_clean'] = df_full['entry_debit_f1'].clip(lower=0)

# Début du débit : 2024-05-03
# Fin de la pluie : 2024-12-31
END_DATE_OVERLAP = '2024-12-31 23:00:00'
df_full = df_full[df_full.index <= END_DATE_OVERLAP]

print(f"Période finale retenue : du {df_full.index.min()} au {df_full.index.max()}")

series = df_full['entry_debit_clean']
peaks_indices, _ = find_peaks(series, height=20, distance=5)
cycle_durations = pd.Series(series.index[peaks_indices]).diff().dropna().dt.total_seconds() / 60

print(f"Durée MOYENNE d'un cycle (2024) : {cycle_durations.mean():.1f} min")
print(f"Durée MAXIMALE d'un cycle (2024) : {cycle_durations.max():.1f} min")
window_size = 180
df_full['entry_debit_smooth_3h'] = df_full['entry_debit_clean'].rolling(window=window_size, center=True).mean()

df_plot = df_full.iloc[::10, :] 

fig_raw = px.line(df_plot, y='entry_debit_clean', 
              title='Débit Brut Nettoyé (Période Mai - Déc 2024)',
              labels={'entry_debit_clean': 'Débit Pompe (m3/h)', 'ts': 'Temps'},
              render_mode='webgl')
fig_raw.update_traces(line=dict(color='orange', width=1))
fig_raw.show()

fig_smooth = px.line(df_plot, y='entry_debit_smooth_3h', 
              title='Target Finale : Débit Lissé sur 3h (Période Mai - Déc 2024)',
              labels={'entry_debit_smooth_3h': 'Débit Moyen (m3/h)', 'ts': 'Temps'},
              render_mode='webgl')
fig_smooth.update_traces(line=dict(color='green', width=2))
fig_smooth.show()

# 6. SAUVEGARDE
output_path = 'bronze/bronze/sensor_P_clean.csv'

cols_to_drop = ['aeration_f1', 'oxygen_f1', 'redox_f1']
df_save = df_full.drop(columns=cols_to_drop, errors='ignore')

print(f"Colonnes conservées : {list(df_save.columns)}")
df_save.to_csv(output_path)

Pic fin Octobre 2024 ? pourquoi ? Beaucoup de pluies?